In [ ]:
import os
import skimage.io
import sys
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

# Root directory of the project
ROOT_DIR = os.path.abspath("./")
# Root directory of the dataset
DATA_DIR = os.path.abspath("../dataset/wad")
sys.path.append(ROOT_DIR)  # To find local version of the library
# Directory to save logs and trained model
LOGS_DIR = os.path.join(ROOT_DIR, "logs")

# Import Mask RCNN
from mrcnn.config import Config
import mrcnn.model as modellib
from mrcnn import visualize

## Configuration

In [ ]:
from project import wad_data
cfg = wad_data.WADConfig()
cfg.display()

dataset = wad_data.WADDataset()
dataset.load_data(DATA_DIR, "train")
dataset.prepare()

## Load Model

In [ ]:
# Create model in inference mode
model = modellib.MaskRCNN(mode="inference", config=cfg,
                          model_dir=LOGS_DIR)

model.load_weights(os.path.join(LOGS_DIR, "wad20180620T1548/mask_rcnn_wad_0001.h5"))

## Inference

In [ ]:
# Load a random image from the dataset
image_id = np.random.randint(0, dataset.num_images)
image = dataset.load_image(image_id)
gt_masks = dataset.load_mask(image_id)

results = model.detect([image], verbose=1)[0]

visualize.display_instances(image, results['rois'], results['masks'], results['class_ids'], 
                            wad_data.classes, results['scores'])